In [1]:
import itertools
import pathlib
import pyam

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as compile

# Catch setting with copy warning
pd.options.mode.chained_assignment = None

sns.set_style('whitegrid')

<IPython.core.display.Javascript object>

# Read Data

In [2]:
datapath = pathlib.Path('../../data')
fulldf = pyam.IamDataFrame(datapath / 'gidden_brutschin_et_al_2023.xlsx')

pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.core - INFO: Reading file ../../data/gidden_brutschin_et_al_2023.xlsx
pyam.core - INFO: Reading meta indicators


In [3]:
scens = pd.read_excel(datapath / 'scenarios_assessed.xlsx', sheet_name='equity_comparison')

Now, we filter out for the necessary scenarios.

In [4]:
equity_data = fulldf.filter(
    scenario=scens['Scenario']
)

In [5]:
native_regions = [
    'AFR',
     'CPA',
     'EEU',
     'FSU',
     'LAM',
     'MEA',
     'NAM',
     'PAO',
     'PAS',
     'SAS',
     'WEU',
     'R5LAM',
     'R5MAF',
     'R5OECD90+EU',
     'R5REF',
     'R5ASIA'
]
equity_data = equity_data.filter(region=native_regions, keep=False)

In [6]:
equity_data.region

['China & Centrally Planned Asia',
 'Developed Regions',
 'Latin America',
 'Middle East & Africa',
 'South & South East Asia',
 'World']

In [7]:
meta = equity_data.meta
meta_blacklist = []
for r in native_regions:
    cols = [c for c in meta.columns if r in c]
    meta_blacklist += cols
equity_data.meta = meta[list(set(meta.columns) - set(meta_blacklist))]

# Equity of carbon budgets

First, we need the cumulative debt to be summed up between 1990 and 2019.

In [8]:
equity_data_debt = (
    equity_data
    .filter(
        variable='Emissions|CO2|Debt',
        year=range(1990,2020)
    )
)
equity_data_debt.head()

,model,scenario,region,variable,unit,year,value
0,MESSAGEix-GLOBIOM_1.1,EN_NPi2020-DACm-MP-median-stor3-final_1000,China & Centrally Planned Asia,Emissions|CO2|Debt,Mt CO2/yr,1990,-4238.023991
1,MESSAGEix-GLOBIOM_1.1,EN_NPi2020-DACm-MP-median-stor3-final_1000,China & Centrally Planned Asia,Emissions|CO2|Debt,Mt CO2/yr,1991,-4151.551031
2,MESSAGEix-GLOBIOM_1.1,EN_NPi2020-DACm-MP-median-stor3-final_1000,China & Centrally Planned Asia,Emissions|CO2|Debt,Mt CO2/yr,1992,-4065.527500
3,MESSAGEix-GLOBIOM_1.1,EN_NPi2020-DACm-MP-median-stor3-final_1000,China & Centrally Planned Asia,Emissions|CO2|Debt,Mt CO2/yr,1993,-3979.933033
4,MESSAGEix-GLOBIOM_1.1,EN_NPi2020-DACm-MP-median-stor3-final_1000,China & Centrally Planned Asia,Emissions|CO2|Debt,Mt CO2/yr,1994,-3894.748473


In [9]:
for region in equity_data_debt.region:
    variable=f'Cumulative Emissions|CO2|Debt in {region} until 2020'
    data_ts = (
        equity_data_debt
        .filter(region=region)
        .timeseries()
    )
    equity_data.set_meta(
        data_ts.apply(
            lambda x: pyam.cumulative(
                x,
                first_year=1990,
                last_year=2019
            ),
            axis=1
        ),
        name=variable
    )

Identify columns we will use for the analysis of the carbon budgets.

In [10]:
cols_debt = [
    x for x in equity_data.meta 
    if x.startswith('Cumulative Emissions|CO2|Debt')
    and not (x.endswith('until net-zero'))
]

In [11]:
cols_fair = [
    x for x in equity_data.meta
    if x.startswith('Cumulative Emissions|CO2|Fair')
]

In [12]:
cols_model = [
    x for x in equity_data.meta
    if x.startswith('Cumulative Emissions|CO2 in')
]

In [13]:
cols_equity = cols_debt + cols_fair + cols_model

In [14]:
fair_budget_data = equity_data.meta[cols_equity]

Finally, we want to add the debt to the forward-looking fair emission allocations. 

In [15]:
for region in equity_data.region:
    a_to_add = f'Cumulative Emissions|CO2|Fair in {region} until net-zero'
    b_to_add = f'Cumulative Emissions|CO2|Debt in {region} until 2020'
    name = f'Cumulative Emissions|CO2|Fair in {region} until net-zero-including-debt'
    # Add the necessary data
    fair_budget_data.loc[:,name] = (
        fair_budget_data.loc[:,a_to_add] 
        - 
        fair_budget_data.loc[:,b_to_add]
    )

Now we need to reshape the data so that we can plot it out.

In [16]:
fair_budget_reshaped = (
    fair_budget_data
    .stack()
    .reset_index(level=-1)
    .rename(
        columns={0:'value'}
    )
)

Add a dictionary to facilitate the renaming.

In [17]:
renaming_dictionary = {
    'China & Centrally Planned Asia':'China',
    'Developed Regions':'Developed',
    'Latin America':'Latin',
    'Middle East & Africa':'Africa',
    'South & South East Asia':'Asia'
}

In [18]:
for key in renaming_dictionary.keys():
    fair_budget_reshaped.loc[:,'level_2'] = (
        fair_budget_reshaped.loc[:,'level_2'].apply(
            lambda x: x.replace(
                key,
                renaming_dictionary[key]
            )
        )
    )

In [19]:
split_elements = (
    fair_budget_reshaped['level_2']
    .apply(lambda x: x.split(' '))
)

In [20]:
fair_budget_reshaped.loc[:,'variable'] = split_elements.apply(lambda x: x[1])
fair_budget_reshaped.loc[:,'region'] = split_elements.apply(lambda x: x[3])
fair_budget_reshaped.loc[:,'consideration'] = split_elements.apply(lambda x: x[5])

In [21]:
fair_budget_reshaped = (
    fair_budget_reshaped[
        fair_budget_reshaped['variable']
        .isin(['Emissions|CO2|Fair','Emissions|CO2'])
    ]
)

Copy the model data and assign the additional consideration type so that we can pivot the data.

In [22]:
fair_budget_reshaped_model = fair_budget_reshaped[fair_budget_reshaped['variable']=='Emissions|CO2']

In [23]:
fair_budget_reshaped_model['consideration'] = 'net-zero-including-debt'

In [24]:
fair_budget_reshaped_model['value'].isnull().any()

False

In [25]:
fair_budget_reshaped = pd.concat(
    [fair_budget_reshaped, fair_budget_reshaped_model]
)

In [26]:
fair_budget_reshaped['value'].isnull().any()

False

In [27]:
fair_budget_final = pd.pivot_table(
    fair_budget_reshaped.drop(columns='level_2').reset_index(),
    index=['model','scenario','region','consideration'],
    columns=['variable'],
    values='value'
)

In [28]:
fair_budget_final

variable                                                                                            Emissions|CO2  \
model                 scenario                                   region    consideration                            
MESSAGEix-GLOBIOM_1.1 EN_NPi2020-DACm-MP-median-stor3-final_1000 Africa    net-zero                 168342.207398   
                                                                           net-zero-including-debt  168342.207398   
                                                                 Asia      net-zero                 234390.778236   
                                                                           net-zero-including-debt  234390.778236   
                                                                 China     net-zero                 217808.798862   
...                                                                                                           ...   
                      EN_NPi2020-stor3-final_700                 Developed net-zero-including-debt  209493.000886   
                                                                 Latin     net-zero                  20718.103007   
                                                                           net-zero-including-debt   20718.103007   
                                                                 World     net-zero                 640603.766001   
                                                                           net-zero-including-debt  640603.766001   

variable                                                                                            Emissions|CO2|Fair  
model                 scenario                                   region    consideration                                
MESSAGEix-GLOBIOM_1.1 EN_NPi2020-DACm-MP-median-stor3-final_1000 Africa    net-zero                      216599.511197  
                                                                           net-zero-including-debt       274798.539020  
                                                                 Asia      net-zero                      310442.610053  
                                                                           net-zero-including-debt       503278.224715  
                                                                 China     net-zero                      165846.347047  
...                                                                                                                ...  
                      EN_NPi2020-stor3-final_700                 Developed net-zero-including-debt      -141403.983385  
                                                                 Latin     net-zero                       53360.362807  
                                                                           net-zero-including-debt        28525.093800  
                                                                 World     net-zero                      640603.766001  
                                                                           net-zero-including-debt       640603.766001  

[132 rows x 2 columns]

In [29]:
fair_budget_final.isnull().any()

variable
Emissions|CO2         False
Emissions|CO2|Fair    False
dtype: bool

In [30]:
fair_budget_final.index.get_level_values('region').unique()

Index(['Africa', 'Asia', 'China', 'Developed', 'Latin', 'World'], dtype='object', name='region')

In [31]:
regions_to_assess = list(
    set(fair_budget_final.index.get_level_values('region').unique()) - set(['World'])
)

In [32]:
fair_budget_final.head()

variable                                                                                         Emissions|CO2  \
model                 scenario                                   region consideration                            
MESSAGEix-GLOBIOM_1.1 EN_NPi2020-DACm-MP-median-stor3-final_1000 Africa net-zero                 168342.207398   
                                                                        net-zero-including-debt  168342.207398   
                                                                 Asia   net-zero                 234390.778236   
                                                                        net-zero-including-debt  234390.778236   
                                                                 China  net-zero                 217808.798862   

variable                                                                                         Emissions|CO2|Fair  
model                 scenario                                   region consideration                                
MESSAGEix-GLOBIOM_1.1 EN_NPi2020-DACm-MP-median-stor3-final_1000 Africa net-zero                      216599.511197  
                                                                        net-zero-including-debt       274798.539020  
                                                                 Asia   net-zero                      310442.610053  
                                                                        net-zero-including-debt       503278.224715  
                                                                 China  net-zero                      165846.347047

In [33]:
fair_budget_final = fair_budget_final.loc[pd.IndexSlice[:, :, regions_to_assess,:]]

In [34]:
if fair_budget_final.isnull().any().any():
    raise ValueError(f'Null columns: {fair_budget_final.isnull().any()}')

Now, we take care of unit conversion issues.

In [35]:
fair_budget_final /= 1e3

In [36]:
fair_budget_final = fair_budget_final.assign(unit='Gt CO2')

In [37]:
fair_budget_final

variable                                                                                         Emissions|CO2  \
model                 scenario                                   region consideration                            
MESSAGEix-GLOBIOM_1.1 EN_NPi2020-DACm-MP-median-stor3-final_1000 China  net-zero                    217.808799   
                                                                        net-zero-including-debt     217.808799   
                      EN_NPi2020-DACm-MP-median-stor3-final_500  China  net-zero                    105.255272   
                                                                        net-zero-including-debt     105.255272   
                      EN_NPi2020-DACm-MP-median-stor3-final_700  China  net-zero                    148.234544   
...                                                                                                        ...   
                      EN_NPi2020-stor3-final_1000                Latin  net-zero-including-debt      -4.289697   
                      EN_NPi2020-stor3-final_500                 Latin  net-zero                     12.763978   
                                                                        net-zero-including-debt      12.763978   
                      EN_NPi2020-stor3-final_700                 Latin  net-zero                     20.718103   
                                                                        net-zero-including-debt      20.718103   

variable                                                                                         Emissions|CO2|Fair  \
model                 scenario                                   region consideration                                 
MESSAGEix-GLOBIOM_1.1 EN_NPi2020-DACm-MP-median-stor3-final_1000 China  net-zero                         165.846347   
                                                                        net-zero-including-debt          211.060488   
                      EN_NPi2020-DACm-MP-median-stor3-final_500  China  net-zero                          83.669203   
                                                                        net-zero-including-debt          128.883344   
                      EN_NPi2020-DACm-MP-median-stor3-final_700  China  net-zero                         117.095190   
...                                                                                                             ...   
                      EN_NPi2020-stor3-final_1000                Latin  net-zero-including-debt           53.369968   
                      EN_NPi2020-stor3-final_500                 Latin  net-zero                          37.406550   
                                                                        net-zero-including-debt           12.571281   
                      EN_NPi2020-stor3-final_700                 Latin  net-zero                          53.360363   
                                                                        net-zero-including-debt           28.525094   

variable                                                                                           unit  
model                 scenario                                   region consideration                    
MESSAGEix-GLOBIOM_1.1 EN_NPi2020-DACm-MP-median-stor3-final_1000 China  net-zero                 Gt CO2  
                                                                        net-zero-including-debt  Gt CO2  
                      EN_NPi2020-DACm-MP-median-stor3-final_500  China  net-zero                 Gt CO2  
                                                                        net-zero-including-debt  Gt CO2  
                      EN_NPi2020-DACm-MP-median-stor3-final_700  China  net-zero                 Gt CO2  
...                                                                                                 ...  
                      EN_NPi2020-stor3-final_1000                Latin  net-zero-including-debt  Gt CO2  
               

In [38]:
fig8_data = (
    pd.merge(
        fair_budget_final.reset_index(),
        scens, 
        right_on=['Scenario'], 
        left_on=['scenario']
    )
    .drop(columns=['Model','Scenario'])
)

In [39]:
fig8_data.loc[:,'region'].unique()

array(['China', 'Developed', 'Africa', 'Asia', 'Latin'], dtype=object)

In [40]:
fig8_data.head()

,model,scenario,region,consideration,Emissions|CO2,Emissions|CO2|Fair,unit,Temperature,DAC,Technoeconomic,Diffusion,Governance,Governance SSP
0,MESSAGEix-GLOBIOM_1.1,EN_NPi2020-DACm-MP-median-stor3-final_1000,China,net-zero,217.808799,165.846347,Gt CO2,C3 with DAC,DAC Included,Medium,Medium,Immediate Global Action,NaN
1,MESSAGEix-GLOBIOM_1.1,EN_NPi2020-DACm-MP-median-stor3-final_1000,China,net-zero-including-debt,217.808799,211.060488,Gt CO2,C3 with DAC,DAC Included,Medium,Medium,Immediate Global Action,NaN
2,MESSAGEix-GLOBIOM_1.1,EN_NPi2020-DACm-MP-median-stor3-final_1000,Developed,net-zero,250.817605,168.181019,Gt CO2,C3 with DAC,DAC Included,Medium,Medium,Immediate Global Action,NaN
3,MESSAGEix-GLOBIOM_1.1,EN_NPi2020-DACm-MP-median-stor3-final_1000,Developed,net-zero-including-debt,250.817605,-89.453671,Gt CO2,C3 with DAC,DAC Included,Medium,Medium,Immediate Global Action,NaN
4,MESSAGEix-GLOBIOM_1.1,EN_NPi2020-DACm-MP-median-stor3-final_1000,Africa,net-zero,168.342207,216.599511,Gt CO2,C3 with DAC,DAC Included,Medium,Medium,Immediate Global Action,NaN


Calculate the deviation from the fair benchmarks.

In [41]:
fig8_data.loc[:,'delta'] = fig8_data['Emissions|CO2|Fair'] - fig8_data['Emissions|CO2']

In [42]:
fig8_data

,model,scenario,region,consideration,Emissions|CO2,Emissions|CO2|Fair,unit,Temperature,DAC,Technoeconomic,Diffusion,Governance,Governance SSP,delta
0,MESSAGEix-GLOBIOM_1.1,EN_NPi2020-DACm-MP-median-stor3-final_1000,China,net-zero,217.808799,165.846347,Gt CO2,C3 with DAC,DAC Included,Medium,Medium,Immediate Global Action,NaN,-51.962452
1,MESSAGEix-GLOBIOM_1.1,EN_NPi2020-DACm-MP-median-stor3-final_1000,China,net-zero-including-debt,217.808799,211.060488,Gt CO2,C3 with DAC,DAC Included,Medium,Medium,Immediate Global Action,NaN,-6.748311
2,MESSAGEix-GLOBIOM_1.1,EN_NPi2020-DACm-MP-median-stor3-final_1000,Developed,net-zero,250.817605,168.181019,Gt CO2,C3 with DAC,DAC Included,Medium,Medium,Immediate Global Action,NaN,-82.636585
3,MESSAGEix-GLOBIOM_1.1,EN_NPi2020-DACm-MP-median-stor3-final_1000,Developed,net-zero-including-debt,250.817605,-89.453671,Gt CO2,C3 with DAC,DAC Included,Medium,Medium,Immediate Global Action,NaN,-340.271276
4,MESSAGEix-GLOBIOM_1.1,EN_NPi2020-DACm-MP-median-stor3-final_1000,Africa,net-zero,168.342207,216.599511,Gt CO2,C3 with DAC,DAC Included,Medium,Medium,Immediate Global Action,NaN,48.257304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,MESSAGEix-GLOBIOM_1.1,EN_NPi2020-stor3-final_700,Africa,net-zero-including-debt,94.707209,201.247441,Gt CO2,C2 without DAC,No DAC,No DAC,No DAC,Immediate Global Action,NaN,106.540232
106,MESSAGEix-GLOBIOM_1.1,EN_NPi2020-stor3-final_700,Asia,net-zero,134.839168,211.264683,Gt CO2,C2 without DAC,No DAC,No DAC,No DAC,Immediate Global Action,NaN,76.425515
107,MESSAGEix-GLOBIOM_1.1,EN_NPi2020-stor3-final_700,Asia,net-zero-including-debt,134.839168,404.100298,Gt CO2,C2 without DAC,No DAC,No DAC,No DAC,Immediate Global Action,NaN,269.261129
108,MESSAGEix-GLOBIOM_1.1,EN_NPi2020-stor3-final_700,Latin,net-zero,20.718103,53.360363,Gt CO2,C2 without DAC,No DAC,No DAC,No DAC,Immediate Global Action,NaN,32.642260


Save this out for plot generation.

In [43]:
fig8_data.to_excel(
    'fig8_data.xlsx'
)